In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims
import pandas as pd
import numpy as np
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module simple_controls
loading module save_resampled
loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


#### To visualize specific 3D volumic sulci for specific subjects

In [2]:
dataset =  'hcp'
region = 'S.C.-sylv.' #"S.C.-sylv." "S.T.s."
side = 'R' #"L"

In [17]:
sample = [102816,
 103010,
 103111,
 103212,
 103414,
 103515,
 103818,
 104012,
 104416,
 104820,
 105014,
 105115,
 105216,
 105620,
 105923]

['sub-3672666', 'sub-2495997']


In [20]:
volume=True
nb_columns=2
block = a.createWindowsBlock(nb_columns) # nb of columns
dic_windows = {}

referential1 = a.createReferential()

mm_skeleton_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'
dic_windows['Sulci_color']=a.loadObject('/casa/host/build/share/brainvisa-share-5.2/nomenclature/hierarchy/sulcal_root_colors.hie')
for i, subject_id in enumerate(sample):
    volume_path = f"{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz"
    
    if os.path.isfile(volume_path):
        vol = aims.read(volume_path)
        
        dic_windows[f'a_vol{nb_columns*i}'] = a.toAObject(vol)
        #dic_windows[f'a_vol{i}'].setPalette(absoluteMode=True)
        dic_windows[f'rvol{nb_columns*i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{nb_columns*i}']], method='VolumeRenderingFusionMethod')
        dic_windows[f'rvol{nb_columns*i}'].releaseAppRef()
        dic_windows[f'rvol{nb_columns*i}'].assignReferential(referential1)
        dic_windows[f'wvr{nb_columns*i}'] = a.createWindow('3D', block=block) #geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
        dic_windows[f'wvr{nb_columns*i}'].addObjects(dic_windows[f'rvol{nb_columns*i}'])
    else:
        print(f"{volume_path} is not a correct path, or the .nii.gz doesn't exist")

    path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject}/t1mri/BL' # be careful HCP here
    white_matter_path = f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject_id}_{side}white.gii'
    sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject_id}.arg'
    spam_labelled_sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/spam_session_auto/{side}{subject_id}_spam_session_auto.arg'
    deep_labelled_sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/deepcnn_session_auto/{side}{subject_id}_deepcnn_session_auto.arg'

    if os.path.isfile(white_matter_path):
        # To visualize the white matter for specific people
        dic_windows[f'white_{subject_id}'] = a.loadObject(white_matter_path)
        #dic_windows[f'white_{subject_id}'].loadReferentialFromHeader()
        dic_windows[f'white_{subject_id}'].assignReferential(referential1)
    else:
        print(f"{white_matter_path} is not a correct path, or the .white.gii doesn't exist")

    #if os.path.isfile(sulci_path):
        # To visualize the sulci for specific people
        #dic_windows[f'sulci_{subject_id}'] = a.loadObject(sulci_path)
        #dic_windows[f'sulci_{subject_id}'].loadReferentialFromHeader()
    #else:
        #print(f"{sulci_path} is not a correct path, or the .arg doesn't exist")
    
    if os.path.isfile(spam_labelled_sulci_path):
        # To visualize the annotated sulci for specific people
        dic_windows[f'sulci_labelled_{subject_id}'] = a.loadObject(spam_labelled_sulci_path)
        #dic_windows[f'sulci_labelled_{subject_id}'].loadReferentialFromHeader()
        dic_windows[f'sulci_labelled_{subject_id}'].assignReferential(referential1)
    else:
        print(f"{spam_labelled_sulci_path} is not a correct path, or the .arg doesn't exist")
        print("Automatic try with 'deepcnn_session_auto' instead of 'spam_session_auto'")
        if  os.path.isfile(deep_labelled_sulci_path):
            # To visualize the annotated sulci for specific people
            dic_windows[f'sulci_labelled_{subject_id}'] = a.loadObject(deep_labelled_sulci_path)
            #dic_windows[f'sulci_labelled_{subject_id}'].loadReferentialFromHeader()
            dic_windows[f'sulci_labelled_{subject_id}'].assignReferential(referential1)
    
    #dic_windows[f'wvr{nb_columns*i+1}'] = a.createWindow('3D', block=block)
    #dic_windows[f'wvr{nb_columns*i+1}'].addObjects([dic_windows2[f'white_{subject_id}'], dic_windows[f'sulci_{subject_id}']])
    dic_windows[f'wvr{nb_columns*i+1}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{nb_columns*i+1}'].addObjects([dic_windows[f'white_{subject_id}'], dic_windows[f'sulci_labelled_{subject_id}']])

nifti transfo: 2
/home/ad279118/tmp/sub-3672666/ses-2/anat/t1mri/default_acquisition/default_analysis/folds/3.1/spam_session_auto/Rsub-3672666_spam_session_auto.arg is not a correct path, or the .arg doesn't exist
Automatic try with 'deepcnn_session_auto' instead of 'spam_session_auto'
memory limit: 45798198476
Reading FGraph version 3.3


bounding box found : 14, 26, 40
                     86, 194, 150
nifti transfo: 3
nifti transfo: 2
/home/ad279118/tmp/sub-2495997/ses-2/anat/t1mri/default_acquisition/default_analysis/folds/3.1/spam_session_auto/Rsub-2495997_spam_session_auto.arg is not a correct path, or the .arg doesn't exist
Automatic try with 'deepcnn_session_auto' instead of 'spam_session_auto'
memory limit: 45819648409
Reading FGraph version 3.1


bounding box found : 14, 27, 42
                     85, 196, 159
nifti transfo: 3


#### To visualize the BUCKETS for specific people 

In [21]:
bucket_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}buckets'

bucket_files = []
bck_path = f'{bucket_path}/{subject_id}_cropped_skeleton.bck'

for subject_id in sample:
    if os.path. isfile(bck_path):
        bucket_files.append(bck_path)
    else:
        print(f"{bck_path} is not a correct path, or the .bck doesn't exist")

for i, file in enumerate(bucket_files):
    dic_windows[f'bck_{i}'] = a.loadObject(file)
    dic_windows[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    dic_windows[f'w_{i}'].addObjects(dic_windows[f'bck_{i}'])

memory limit: 45595076198
Reading FGraph version 3.1


bounding box found : 16, 34, 49
                     88, 193, 156
nifti transfo: 1
nifti transfo: 1
memory limit: 45615969075
Reading FGraph version 3.1


bounding box found : 16, 36, 34
                     87, 208, 152
nifti transfo: 2
memory limit: 45611702681
Reading FGraph version 3.1


bounding box found : 16, 25, 35
                     95, 195, 150
nifti transfo: 1
nifti transfo: 1
memory limit: 45612046745
Reading FGraph version 3.1


bounding box found : 20, 33, 47
                     89, 204, 153
nifti transfo: 2
nifti transfo: 1
memory limit: 45606014156
Reading FGraph version 3.1


...

bounding box found : 19, 37, 39
                     95, 217, 150
nifti transfo: 2
nifti transfo: 2
memory limit: 45576975155
Reading FGraph version 3.1


bounding box found : 14, 21, 38
                     88, 176, 144
nifti transfo: 3
nifti transfo: 2
memory limit: 45596170649
Reading FGraph version 3.1


bounding box found : 15, 19, 42
                     88, 176, 150
nifti transfo: 3
nifti transfo: 1
memory limit: 45619894681
Reading FGraph version 3.1


bounding box found : 25, 35, 55
                     101, 202, 172
nifti transfo: 2


nifti transfo: 1
nifti transfo: 1
nifti transfo: 2
nifti transfo: 2
nifti transfo: 1
nifti transfo: 3
nifti transfo: 2
nifti transfo: 3
nifti transfo: 1
nifti transfo: 2
nifti transfo: 2
nifti transfo: 2
nifti transfo: 1
nifti transfo: 1
nifti transfo: 2
nifti transfo: 2
nifti transfo: 1
nifti transfo: 3
no position could be read at 221, 143
no position could be read at 197, 90
no position could be read at 209, 55
Position : -21.0162, -73.5132, -14.4223, 0
Position : -44.6912, -50.1271, -25.3672, 0
Position : -34.7306, 4.10713, -47.6484, 0
no position could be read at 173, 87
Position : 47.3809, 173.271, 52.9856, 0
Position : 42.7566, 142.728, 52.256, 0
no position could be read at 161, 136
Position : 38.034, 156.439, 47.4465, 0
Position : -61.719, 3.55056, -40.395, 0
Position : -41.341, -33.8254, -24.6045, 0
Position : -44.9775, 68.3375, -44.7286, 0
Position : -59.4141, -20.2527, -45.5596, 0
Position : -46.4447, 42.342, -77.61, 0
no position could be read at 181, 95
Position : 42.2152